In [11]:
import json
import os

data = []

with open('../../Datasets/Experiment_1_one_way_communication_10_minute_input_sample.json', 'r') as file:
    for line in file:
        data.append(json.loads(line))

print(data)


[{'No.': 1, 'Time': 0.0, 'Source': '0x1de6', 'Destination': 'Broadcast', 'Protocol': 'ZigBee', 'Length': 80, 'Info': 'Link Status', 'Info_clean': 'Link Status'}, {'No.': 2, 'Time': 8.041278, 'Source': '0x1de6', 'Destination': '0xd7a7', 'Protocol': 'ZigBee HA', 'Length': 69, 'Info': 'ZCL: Read Attributes Response, Seq: 216', 'Info_clean': 'ZCL: Read Attributes Response,'}, {'No.': 3, 'Time': 8.073592, 'Source': '0x1de6', 'Destination': '0xd7a7', 'Protocol': 'ZigBee HA', 'Length': 61, 'Info': 'ZCL: Read Attributes Response, Seq: 217', 'Info_clean': 'ZCL: Read Attributes Response,'}, {'No.': 4, 'Time': 14.242098, 'Source': '0x1de6', 'Destination': 'Broadcast', 'Protocol': 'ZigBee', 'Length': 80, 'Info': 'Link Status', 'Info_clean': 'Link Status'}, {'No.': 5, 'Time': 20.786429, 'Source': '0x1de6', 'Destination': '0xd7a7', 'Protocol': 'ZigBee HA', 'Length': 69, 'Info': 'ZCL: Read Attributes Response, Seq: 239', 'Info_clean': 'ZCL: Read Attributes Response,'}, {'No.': 6, 'Time': 20.812187, '

In [12]:
Source_Type = set(item['Source'] for item in data)
num_unique_src = len(Source_Type)

Sender = []
for src in Source_Type:
    Sender.append(src)
    print(src)

print("Senders: ", Sender) 

0x1de6
Senders:  ['0x1de6']


In [13]:
Destination_Type = set(item['Destination'] for item in data)
num_unique_dst = len(Destination_Type)

Receiver = []
for dst in Destination_Type:
    Receiver.append(dst)
    print(dst)

print("Receivers: ", Receiver) 

0xd7a7
Broadcast
Receivers:  ['0xd7a7', 'Broadcast']


In [14]:
""" Change Broadcast with the address """
for item in data:
    if item['Destination'] == 'Broadcast':
        item['Destination'] = '0xfffc'

In [15]:
Destination_Type = set(item['Destination'] for item in data)
num_unique_dst = len(Destination_Type)

Receiver = []
for dst in Destination_Type:
    Receiver.append(dst)
    print(dst)

print("Receivers: ", Receiver) 

0xd7a7
0xfffc
Receivers:  ['0xd7a7', '0xfffc']


In [16]:
Protocol_Type = set(item['Protocol'] for item in data)
num_unique_protocol = len(Protocol_Type)

Protocols = []
for protocols in Protocol_Type:
    Protocols.append(protocols)
    print(protocols)

print("Protocols: ", Protocols) 

ZigBee
ZigBee HA
Protocols:  ['ZigBee', 'ZigBee HA']


In [17]:
""" Create sample prompt from json file """
Sample_Packets= []

for item in data:
    sample_packets = {
        'time': str(item['Time']),
        'src': item['Source'],
        'dst': item['Destination'],
        'protocol': item['Protocol'],
        'length': str(item['Length']),
        'info': item['Info']
    }
    Sample_Packets.append(sample_packets)
    #print(sample_prompt)
print(Sample_Packets)

[{'time': '0.0', 'src': '0x1de6', 'dst': '0xfffc', 'protocol': 'ZigBee', 'length': '80', 'info': 'Link Status'}, {'time': '8.041278', 'src': '0x1de6', 'dst': '0xd7a7', 'protocol': 'ZigBee HA', 'length': '69', 'info': 'ZCL: Read Attributes Response, Seq: 216'}, {'time': '8.073592', 'src': '0x1de6', 'dst': '0xd7a7', 'protocol': 'ZigBee HA', 'length': '61', 'info': 'ZCL: Read Attributes Response, Seq: 217'}, {'time': '14.242098', 'src': '0x1de6', 'dst': '0xfffc', 'protocol': 'ZigBee', 'length': '80', 'info': 'Link Status'}, {'time': '20.786429', 'src': '0x1de6', 'dst': '0xd7a7', 'protocol': 'ZigBee HA', 'length': '69', 'info': 'ZCL: Read Attributes Response, Seq: 239'}, {'time': '20.812187', 'src': '0x1de6', 'dst': '0xd7a7', 'protocol': 'ZigBee HA', 'length': '61', 'info': 'ZCL: Read Attributes Response, Seq: 240'}, {'time': '28.563272', 'src': '0x1de6', 'dst': '0xfffc', 'protocol': 'ZigBee', 'length': '80', 'info': 'Link Status'}, {'time': '33.56803', 'src': '0x1de6', 'dst': '0xd7a7', 'p

In [18]:
len(Sample_Packets)

128

# TRAFFIC GENERATION GPT 5 - EXP 1

In [19]:
Sources = ["0x1de6"]
Destinations = ["0xd7a7", "0xfffc"]
Protocols = ["ZigBee", "ZigBee HA"]

In [ ]:
import openai
import os
import time
import pandas as pd
import numpy as np

N = [1,2,3,4,5,6,7,8,9,10] # number of trial
computation_time = []
input_tokens = []
output_tokens = []
total_tokens = []


client = openai.OpenAI(api_key=os.environ["OPENAI_KEY"])
prompt = f"""
Examine the following sample ZigBee network traffic data and generate new traffic entries in a similar format.

Sample data: {Sample_Packets}

Rules:
- The format must match the sample exactly, including fields: 'time', 'src', 'dst', 'protocol', 'length', and 'info'.
- The field 'time' is given in seconds format.
- Use src addresses from this list: {Sources}
- Use dst addresses from this list: {Destinations}
- Use one of the following protocols: {Protocols}
- Return the generated json in markdown marker ```json ```

Return output as JSON in the same format, generating new packets of 600 seconds starting from time = 0.
"""


for n in N:
    # Measure start time
    start_time = time.time()
    response = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "system", "content": "You are a realistic ZigBee network traffic generator. You will generate realistic synthetic ZigBee network traffic packets."},
            {"role": "user", "content": prompt}
        ],
        max_completion_tokens=128000     
        )
    # Measure end time
    end_time = time.time()

    # Compute elapsed time
    elapsed_time = end_time - start_time
    print(response.choices[0].message.content)

    print("Input tokens used:", response.usage.prompt_tokens)
    print("Output tokens generated:", response.usage.completion_tokens)
    print("Total tokens used:", response.usage.total_tokens)
    print(f"Computation time: {elapsed_time:.2f} seconds")
    
    computation_time.append(elapsed_time)
    input_tokens.append(response.usage.prompt_tokens)
    output_tokens.append(response.usage.completion_tokens)
    total_tokens.append(response.usage.total_tokens)
    
    generated_message = response.choices[0].message.content
    generated_message = generated_message[8:-4]
    num_samples = len(generated_message.split('\n'))
    print("Number of generated packages:", num_samples)
    
    file_path = fr"../../Generated_Traffic/GPT5_Exp1_Trial_{n}_one_way_communication_10_minute_generated_message.json"

    with open(file_path, "w") as f:
        json.dump(generated_message, f, indent=2)
        
    time.sleep(60)
    
df = pd.DataFrame({
    "Trial": N,   # 1..10
    "Computation_Time": computation_time,
    "Input_Tokens": input_tokens,
    "Output_Tokens": output_tokens,
    "Total_Tokens": total_tokens
})

print(df)
results_array = df.to_numpy()
df.to_csv("../../Generated_Traffic/GPT5_Exp1_results.csv", index=False)


```json
[
  {"time": "0.000000", "src": "0x1de6", "dst": "0xfffc", "protocol": "ZigBee", "length": "80", "info": "Link Status"},
  {"time": "7.804000", "src": "0x1de6", "dst": "0xd7a7", "protocol": "ZigBee HA", "length": "69", "info": "ZCL: Read Attributes Response, Seq: 64"},
  {"time": "7.836000", "src": "0x1de6", "dst": "0xd7a7", "protocol": "ZigBee HA", "length": "61", "info": "ZCL: Read Attributes Response, Seq: 65"},
  {"time": "15.237000", "src": "0x1de6", "dst": "0xfffc", "protocol": "ZigBee", "length": "80", "info": "Link Status"},
  {"time": "20.441000", "src": "0x1de6", "dst": "0xd7a7", "protocol": "ZigBee HA", "length": "69", "info": "ZCL: Read Attributes Response, Seq: 66"},
  {"time": "20.473000", "src": "0x1de6", "dst": "0xd7a7", "protocol": "ZigBee HA", "length": "61", "info": "ZCL: Read Attributes Response, Seq: 67"},
  {"time": "30.481000", "src": "0x1de6", "dst": "0xfffc", "protocol": "ZigBee", "length": "80", "info": "Link Status"},
  {"time": "33.114000", "src": "0